In [1]:
import time
import copy
from typing import List

from nltk.tokenize import word_tokenize
from tqdm import tqdm
from gensim.models import KeyedVectors
import torch
from torch.utils.data import Dataset
from torch import nn
import torch.nn.functional as F
import torch.optim as optimizer
import matplotlib.pyplot as plt
import gensim.downloader

import numpy as np

def get_embed():
    try:
        embed = KeyedVectors.load("word2vec200.kv")
    except:
        embed = gensim.downloader.load('glove-wiki-gigaword-200')
        embed.save("word2vec200.kv")
    return embed



c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embed = get_embed()


In [3]:
embed["hi"].shape

(200,)

In [4]:
def get_word_db():
    with open("words.txt", "r", encoding="utf-8") as f:
        data = f.read()
        data = data.lower()
        data = word_tokenize(data)
        tokens = []
        for token in data:
            if token in embed:
                tokens.append(token)
        return list(tokens)

words = get_word_db()

In [5]:
from annoy import AnnoyIndex
import random

f = embed[words[0]].shape[0]  # Length of item vector that will be indexed

t = AnnoyIndex(f, 'angular')
for i, word in enumerate(words):
    v = embed[word]
    t.add_item(i, v)

index_to_word = dict(enumerate(words))
word_to_index = dict([(word, i) for i, word in enumerate(words)])

t.build(10) # 10 trees
t.save('test.ann')

# ...

word = "machine"
this_word_index = word_to_index[word]

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file
nearest_i = u.get_nns_by_item(this_word_index, 1000) # will find the 1000 nearest neighbors
nearest_word = [(i, index_to_word[i]) for i in nearest_i]
nearest_word

ModuleNotFoundError: No module named 'annoy'